# Creating a CSV agent
By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. 
Agents are systems that use an LLM as a reasoning enginer to determine which actions to take and what the inputs to those actions should be. 
The results of those actions can then be fed back into the agent and it determine whether more actions are needed, or whether it is okay to finish.

In [4]:
# Import relevant functionality
import os
import openai
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = os.getenv("AZURE_OPENAI_VERSION")
openai.api_base = os.getenv('OPENAI_API_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")



In [9]:
# Create the Azure OpenAI model
llm = AzureChatOpenAI(
    deployment_name="gpt4o",  # replace with your deployment name
    temperature=0.5,
    max_tokens=100,
    openai_api_version="2023-03-15-preview"
)

# Define the web_search tool
search = TavilySearchResults(max_results=2)
# Define the Python code execution tool
python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)


# Create the agent with memory and tools
tools = [search, repl_tool]
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# Configuration for conversation thread
config = {"configurable": {"thread_id": "abc123"}}



In [33]:
# # testing the agent with a few messages

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="load the titanic.csv into a pandas dataframe")]}, config
):   
        #print(chunk)
        try:
            if 'agent' in chunk:
                messages = chunk['agent']['messages'][0].content
                print('agent response')
                print(messages)
            elif 'tools' in chunk:
                messages = chunk['tools']['messages'][0].content
                print('tool response')
                print(messages)
            else:
                messages = []
        except KeyError:
            print("KeyError")
    

agent response

tool response
   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Cabin Embarked  
0   330911   7.8292   NaN        Q  
1   363272   7.0000   NaN        S  
2   240276   9.6875   NaN        Q  
3   315154   8.6625   NaN        S  
4  3101298  12.2875   NaN        S  

agent response
The Titanic dataset has been successfull

In [36]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="how many rows are there in the titanic.csv?")]}, config
):
    #print(chunk)
        try:
            if 'agent' in chunk:
                messages = chunk['agent']['messages'][0].content
                print('agent response')
                print(messages)
            elif 'tools' in chunk:
                print(chunk)
                messages = chunk['tools']['messages'][0].content
                print('tool response')
                print(messages)
            else:
                messages = []
        except KeyError:
            print("KeyError")

agent response

{'tools': {'messages': [ToolMessage(content='418\n', name='python_repl', tool_call_id='call_ZtEsL7igpxTrdy4Z89xd2A4Y')]}}
tool response
418

agent response
The Titanic dataset contains 418 rows.
